## Search angles in molecular graph
Find shortest path between two atoms. 

- If the shortest path length is 2 --> there is one atom between interacting partners, calculate angle. 
- If the shortest path length is 3 --> there are two atoms between the partners, calculate dihedral.

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import ase
import ase.visualize
from scipy.spatial import cKDTree as KDTree
from tqdm.notebook import tqdm

In [18]:
dataset = "../champs-scalar-coupling/"
train = pd.read_csv(dataset + "train.csv")
test = pd.read_csv(dataset + "test.csv")
structures = pd.read_csv(dataset + "structures.csv")

In [19]:
bond_lengths = {'H': np.array([0.32, np.NaN, np.NaN]),
                'C': np.array([.75, .67, .60]),
                'O': np.array([.63, .57, .54]),
                'N': np.array([.71, .60, .54]),
                'F': np.array([.64, .59, .53])}

In [20]:
structures['possible_radii'] = [bond_lengths[a] for a in structures.atom]
display(structures.head())

,molecule_name,atom_index,atom,x,y,z,possible_radii
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,"[0.75, 0.67, 0.6]"
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,"[0.32, nan, nan]"
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,"[0.32, nan, nan]"
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,"[0.32, nan, nan]"
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,"[0.32, nan, nan]"


In [21]:
def get_structure(molecule_name):
    return structures[structures['molecule_name'] == molecule_name]

In [22]:
def get_structure_x(structure):
    positions = structure[['x', 'y', 'z']].values
    structure_X = pd.merge(structure, structure, how='outer', on=['molecule_name'])
    structure_X['distance'] = np.linalg.norm(structure_X[['x_x', 'y_x', 'z_x']].values - structure_X[['x_y', 'y_y', 'z_y']].values, axis=1)
    structure_X = structure_X[(structure_X.atom_index_x > structure_X.atom_index_y)]
    structure_X['cutoff'] = (structure_X.distance - (structure_X.possible_radii_x + structure_X.possible_radii_y)).apply(abs).apply(min)
    structure_X.sort_values('cutoff', inplace=True)
    return structure_X

In [23]:
def get_graph(structure, cutoff=0.2):
    structure_x = get_structure_x(structure)
    structure_x = structure_x[~((structure_x.atom_x == 'H') & 
                                (structure_x.atom_y == 'H'))]
    g = nx.Graph()
    positions = structure[['x', 'y', 'z']].values
    g.add_nodes_from(structure['atom_index'], pos=positions)
    
    cols = ['atom_index_x', 'atom_index_y', 'distance', 'cutoff']
    X = [ tuple(i) for i in structure_x[cols].values ]
    
    for i_a, i_b, d, c in X:
        if c < cutoff:
            g.add_edge(i_a, i_b, length=d)
        elif nx.number_connected_components(g) == 1:
            break
        else:
            g.add_edge(i_a, i_b, length=d)
    return g

In [24]:
def cosinus(x0, x1, x2):
    e0 = (x0-x1)
    e1 = (x2-x1)
    e0 = (e0 / np.linalg.norm(e0))
    e1 = (e1 / np.linalg.norm(e1))
    cosinus = np.dot(e0, e1)
    return np.round(cosinus, 5)

In [25]:
def dihedral(x0, x1, x2, x3):

    b0 = -1.0 * (x1 - x0)
    b1 = x2 - x1
    b2 = x3 - x2

    b0xb1 = np.cross(b0, b1)
    b1xb2 = np.cross(b2, b1)

    b0xb1_x_b1xb2 = np.cross(b0xb1, b1xb2)

    y = np.dot(b0xb1_x_b1xb2, b1)*(1.0/np.linalg.norm(b1))
    x = np.dot(b0xb1, b1xb2)
    
    grad = np.arctan2(y, x)
    return grad

In [26]:
def get_relations(structure, g, atom_index_0, atom_index_1):

    shortest_path = nx.shortest_path(g, atom_index_0, atom_index_1)
    shortest_path_atoms = ''.join([structure[structure.atom_index == i].atom.values[0] for i in shortest_path[1:-1]])
    shortest_path_n_bonds = len(shortest_path)-1
    
    cos = None
    dihe = None
    
    if shortest_path_n_bonds == 2:
        x0 = structure[structure.atom_index == shortest_path[0]][['x', 'y', 'z']].values[0]
        x1 = structure[structure.atom_index == shortest_path[1]][['x', 'y', 'z']].values[0]
        x2 = structure[structure.atom_index == shortest_path[2]][['x', 'y', 'z']].values[0]
        cos = cosinus(x0, x1, x2)
        
    if shortest_path_n_bonds == 3:
        x0 = structure[structure.atom_index == shortest_path[0]][['x', 'y', 'z']].values[0]
        x1 = structure[structure.atom_index == shortest_path[1]][['x', 'y', 'z']].values[0]
        x2 = structure[structure.atom_index == shortest_path[2]][['x', 'y', 'z']].values[0]
        x3 = structure[structure.atom_index == shortest_path[3]][['x', 'y', 'z']].values[0]
        dihe = dihedral(x0, x1, x2, x3)
        
    results = {
        'molecule_name': structure.molecule_name.values[0],
        'atom_index_0': atom_index_0,
        'atom_index_1': atom_index_1,
        'shortest_path_atoms': shortest_path_atoms,
        'shortest_path_n_bonds': shortest_path_n_bonds,
        'cosinus': cos,
        'dihedral': dihe
               }
    
    return pd.DataFrame(results, index=[0])

In [27]:
def process(args):
    molecule_name = args['molecule_name']
    structure = get_structure(molecule_name)
    g = get_graph(structure)
    ndxs = zip(args['atom_index_0s'], args['atom_index_1s'])
    return pd.concat([get_relations(structure, g, i, j) for i,j in ndxs])

In [28]:
train = train[['molecule_name', 'atom_index_0', 'atom_index_1']]
train

,molecule_name,atom_index_0,atom_index_1
0,dsgdb9nsd_000001,1,0
1,dsgdb9nsd_000001,1,2
2,dsgdb9nsd_000001,1,3
3,dsgdb9nsd_000001,1,4
4,dsgdb9nsd_000001,2,0
...,...,...,...
4659071,dsgdb9nsd_133884,17,4
4659072,dsgdb9nsd_133884,17,5
4659073,dsgdb9nsd_133884,17,6
4659074,dsgdb9nsd_133884,17,7


In [ ]:
i = 0
train_angles = []
while i < 5000:
    for molecule_name, train in tqdm(train.groupby('molecule_name')):
        ang = process({'molecule_name': molecule_name, 
                       'atom_index_0s': train.atom_index_0.values, 
                       'atom_index_1s': train.atom_index_1.values})
        train_angles.append(ang)
        i += 1

train_angles = pd.concat(train_angles)
train_angles

In [ ]:
test_angles = []
for molecule_name, train in tqdm(test.groupby('molecule_name')):
    ang = process({'molecule_name': molecule_name, 
                   'atom_index_0s': train.atom_index_0.values, 
                   'atom_index_1s': train.atom_index_1.values})
    test_angles.append(ang)

test_angles = pd.concat(test_angles)
test_angles

In [15]:
# import time
# from multiprocessing import Process, Pool, Manager, cpu_count
# cpu_count()

# pool = Pool(processes=4)
# m = Manager()
# q = m.Queue()

# args = []
# start = time.time()

# for molecule_name, train in train.groupby('molecule_name'):
#     args.append({'molecule_name': molecule_name,
#                  'atom_index_0s': train.atom_index_0.values,
#                  'atom_index_1s': train.atom_index_1.values})

# results = pool.map_async(process, args)

# pool.close()
# pool.join()
# end = time.time()

# print(end - start)
# print('Run time:', np.round((end - start) / 60 / 60, 2), 'h')

# result = pd.concat(results.get())
# result.to_csv('angles.csv', index=False)

# print(len(result))
# display(result)

In [16]:
train_angles.to_csv("train_angles.csv")
test_angles.to_csv("test_angles.csv")